In [12]:
import time

import numpy as np
import scipy
import matplotlib.pyplot as plt

In [1]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS
from tenpy.algorithms.exact_diag import ExactDiag

In [2]:
from tenpy.networks.site import SpinSite, SpinHalfSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel

In [4]:
class ClusterIsing(CouplingModel):
    def __init__(self, model_params):
        # Read out/set default parameters
        name = "Cluster Ising model"
        L = model_params.get('L', 5)
        B = model_params.get('B', 0)
        bc_MPS = model_params.get('bc_MPS', 'finite')
        
        # sites
        site = SpinHalfSite(conserve=None)

        # lattice
        bc = 'open'
        lat = Chain(L, site, bc=bc, bc_MPS=bc_MPS)

        # initialize CouplingModel
        CouplingModel.__init__(self, lat)

        # add terms of the Hamiltonian
        self.add_onsite(-B, 0, 'Sigmax')
        self.add_multi_coupling(
            -1,
            [
                ('Sigmaz', -1, 0),
                ('Sigmax', 0, 0),
                ('Sigmaz', 1, 0),
            ]
        )

        # initialize H_MPO
        MPOModel.__init__(self, lat, self.calc_H_MPO())

In [5]:
B_params = np.linspace(0, 2, 21)

In [6]:
B_params

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])

In [27]:
NUM_SITES = 14
MAX_SIZE = 2**30

In [18]:
import h5py
from tenpy.tools import hdf5_io

In [20]:
import os

In [28]:
os.listdir(r"../../data/transverse_cluster_14_site_ed/")

['0_00.h5']

In [29]:
times = list()

for B in B_params:
    print("Commencing B={:.2f} model".format(B))
    start = time.time()
    model=ClusterIsing({'B': B, 'L':NUM_SITES})
    
    psi = MPS.from_desired_bond_dimension(model.lat.mps_sites(), 1, bc=model.lat.bc_MPS)
    psi.canonical_form()

    ED = ExactDiag(model, max_size=MAX_SIZE)

    ED.build_full_H_from_mpo()
    ED.full_diagonalization()
    E0_ED, psi_ED = ED.groundstate()

    data = {
        "wavefunction": psi_ED,
        "energy": E0_ED,
        "paramters": {"B": B}
    }

    filename = r"/home/kieranc/Desktop/numerical_spt_classification/code/data/transverse_cluster_14_site_ed/{:.2f}".format(B).replace(".", "_")
    filename += ".h5"
    with h5py.File(filename, 'w') as f:
        hdf5_io.save_to_hdf5(f, data)

    end = time.time()
    duration = end - start
    times.append(duration)
    print(f"Time taken in minutes: {duration/60}")

Commencing B=0.00 model
Time taken in minutes: 9.075781925519307
Commencing B=0.10 model
Time taken in minutes: 9.11449093023936
Commencing B=0.20 model
Time taken in minutes: 9.212719933191936
Commencing B=0.30 model
Time taken in minutes: 8.694939279556275
Commencing B=0.40 model
Time taken in minutes: 8.683763066927591
Commencing B=0.50 model
Time taken in minutes: 8.622125013669331
Commencing B=0.60 model
Time taken in minutes: 8.92505093018214
Commencing B=0.70 model
Time taken in minutes: 8.763870120048523
Commencing B=0.80 model
Time taken in minutes: 8.672255909442901
Commencing B=0.90 model
Time taken in minutes: 8.629004184405009
Commencing B=1.00 model
Time taken in minutes: 8.687012441953023
Commencing B=1.10 model
Time taken in minutes: 8.687918782234192
Commencing B=1.20 model
Time taken in minutes: 8.650095609823863
Commencing B=1.30 model
Time taken in minutes: 8.691623377799989
Commencing B=1.40 model
Time taken in minutes: 8.687895635763804
Commencing B=1.50 model
Tim